In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.sparse

In [2]:
import os, sys
file_path = '/media/thijs/shoal/Zebrafish_data/spontaneous_data_guillaume/'

In [3]:
# sys.path.append()

In [4]:
list_files = os.listdir(file_path)

In [9]:
list_files

['20180706_Run04_spontaneous_rbm0.h5',
 '20180706_Run04__spikes_only.npy',
 '20180911_Run01_rbm1.h5',
 '20180911_Run01__spikes_only.npy',
 '20180912_Run01_spontaneous_rbm2.h5',
 '20180912_Run01__spikes_only.npy',
 '20180913_Run01_rbm3.h5']

In [10]:
i_list = 6

In [11]:
h5view = h5py.File(file_path + list_files[i_list], 'r')

In [12]:
def get_datasets(data_path, obj):
    if isinstance(obj, h5py.Dataset):  # obj is a dataset
        tmp_name = data_path.split('/')[-1].lower()
        all_keys[tmp_name] = obj.name

all_keys = dict()
h5view.visititems(get_datasets)

In [13]:
all_keys

{'baseline': '/Data/Brain/Analysis/Baseline',
 'dff': '/Data/Brain/Analysis/DFF',
 'drifts': '/Data/Brain/Analysis/Drifts',
 'inferredspikes': '/Data/Brain/Analysis/InferredSpikes',
 'thresholdedspikes': '/Data/Brain/Analysis/ThresholdedSpikes',
 'coordinates': '/Data/Brain/Coordinates',
 'ljpcoordinates': '/Data/Brain/LJPCoordinates',
 'labels': '/Data/Brain/Labels',
 'segmentation': '/Data/Brain/Pixels/Segmentation',
 'temporalmean': '/Data/Brain/Pixels/TemporalMean',
 'rawsignal': '/Data/Brain/RawSignal',
 'timedelays': '/Data/Brain/TimeDelays',
 'times': '/Data/Brain/Times',
 'zbraincoordinates': '/Data/Brain/ZBrainCoordinates',
 'trace': '/Data/Stimulus/Spontaneous/Trace',
 'metadata': '/Metadata'}

In [14]:
key = 'zbraincoordinates'
h5view[all_keys[key]]

<HDF5 dataset "ZBrainCoordinates": shape (3, 44456), type "<f4">

In [15]:
key = 'thresholdedspikes'
h5view[all_keys[key]]

<HDF5 dataset "ThresholdedSpikes": shape (5000, 44456), type "|u1">

In [16]:
# zebrafish coordinates coordinates to use (use zbrain space)
labels = h5view[all_keys['labels']][:, :].T
inds_wb = np.unique(scipy.sparse.find(labels[:, np.arange(294)])[0])
print(f'# (zbrain) wb neurons: {len(inds_wb)}')
print(f'# total neurons: {h5view[all_keys["zbraincoordinates"]].shape[1]}')
# coordinates = rec.zbrain_coords[inds_wb, :]

# (zbrain) wb neurons: 43439
# total neurons: 44456


In [17]:
spikes_data = h5view[all_keys['thresholdedspikes']][:, :]
assert spikes_data.shape[1] == h5view[all_keys["zbraincoordinates"]].shape[1]
spikes_data = spikes_data[:, inds_wb]
spikes_data.shape

(5000, 43439)

In [18]:

np.save(arr=spikes_data, allow_pickle=True,
        file=file_path + list_files[i_list].split('_')[0] + '_' + list_files[i_list].split('_')[1] + '__spikes_only')

In [21]:
fp = file_path + list_files[i_list].split('_')[0] + '_' + list_files[i_list].split('_')[1] + '__spikes_only'

In [22]:
tmp = np.load(fp + '.npy')

In [26]:
tmp.sum(1)

array([12916,  3591,  2890, ...,    41,    25,     2], dtype=uint64)

In [15]:
tmp = None
spikes_data = None

In [75]:
h5view.close()